In [2]:
from DataManipulation.DataHandler import getOHLC_pickle
data = getOHLC_pickle("EURUSD_M_2010_2021.pkl")
data = data.resample('1H').agg({'Open': 'first', 
                        'High': 'max', 
                        'Low': 'min', 
                        'Close': 'last'}).dropna()

In [3]:
#generates the indicators
from DataManipulation.indicators import DEMA, D1, MaxMinRollNorm
demaperiod = [9]
for p in demaperiod:
    data['dema'+str(p)] = DEMA(data['Close'],p)
    data['D1dema'+str(p)] = D1(data['dema'+str(p)])
#    data['normD1dema'+str(p)] = MaxMinRollNorm(data['D1dema'+str(p)], 24*250) # 250 approximated larger than all trading days in a year

stdperiod = [9,100,300]
for p in stdperiod:
    data['std'+str(p)+"_dema9"]  = data['dema9'].rolling(p).std()
    data['std'+str(p)+"_D1dema9"]= data['D1dema9'].rolling(p).std()
print(data.columns)

Index(['Open', 'High', 'Low', 'Close', 'dema9', 'D1dema9', 'std9_dema9',
       'std9_D1dema9', 'std100_dema9', 'std100_D1dema9', 'std300_dema9',
       'std300_D1dema9'],
      dtype='object')


In [4]:
#import numpy as np
#import pandas as pd
#x = pd.Series([np.nan,1,2,3,4,5,6,7,8,9])
#y = pd.Series([np.nan,2,3,4,5,6,7,8,9,10])
#print((x[1:]-y[:-1]).shape)
#print((x[1:]-y[:-1]))
#print((x[1:].to_numpy()-y[:-1].to_numpy()))

In [5]:
import pandas as pd
prediction_ind = ['D1dema9','std9_dema9','std9_D1dema9', 'std100_dema9', 'std100_D1dema9', 'std300_dema9','std300_D1dema9']
preds = pd.DataFrame()
for n in prediction_ind:
    preds[n+"_norm"] = MaxMinRollNorm(data[n], 24*250)
preds.tail()

,D1dema9_norm,std9_dema9_norm,std9_D1dema9_norm,std100_dema9_norm,std100_D1dema9_norm,std300_dema9_norm,std300_D1dema9_norm
Date,,,,,,,
2021-09-30 19:00:00,0.549541,0.035342,0.157625,0.417056,0.216130,0.387081,0.228713
2021-09-30 20:00:00,0.509068,0.031738,0.166833,0.421059,0.216888,0.390208,0.211806
2021-09-30 21:00:00,0.593398,0.035424,0.145746,0.424265,0.215617,0.392990,0.210738
2021-09-30 22:00:00,0.599056,0.033126,0.123020,0.426708,0.215817,0.395560,0.211329
2021-09-30 23:00:00,0.585908,0.033670,0.104754,0.428453,0.214252,0.397978,0.211484


In [6]:
#import matplotlib.pyplot as plt
#plt.plot(preds['D1dema9_norm'].iloc[:]) # might consider to only divide, not subtract minimum as direction might be useful?
#plt.show()
#plt.plot(preds['std9_dema9_norm'].iloc[-500:])
#plt.show()
#plt.plot(preds['std100_dema9_norm'].iloc[-1000:])
#plt.show()

In [7]:
from numpy.lib.stride_tricks import sliding_window_view
raw =  preds.dropna().to_numpy()
wind = sliding_window_view(raw,window_shape = 3*5*24, axis=0)
#print(wind[-1,:])
print(wind.shape)
#print(wind[~pd.isna(wind)].shape)
x = wind[:-1,:]
target_column = "std100_dema9_norm"
y = wind[1:,3,-1]
print(x.shape,y.shape)

(66390, 7, 360)
(66389, 7, 360) (66389,)


In [8]:
from torch.utils.data import TensorDataset
import torch
train_x = torch.Tensor(x[0:int(len(x)*0.7),])
train_y = torch.Tensor(y[0:int(len(x)*0.7)]).unsqueeze(1)
test_x  = torch.Tensor(x[int(len(x)*0.7):,])
test_y  = torch.Tensor(y[int(len(x)*0.7):]).unsqueeze(1)

train_dataset = TensorDataset(torch.Tensor(x[0:int(len(x)*0.7),]),torch.Tensor(y[0:int(len(x)*0.7)]).unsqueeze(1) )
test_dataset =  TensorDataset(torch.Tensor(x[int(len(x)*0.7):,]), torch.Tensor(y[int(len(x)*0.7):]).unsqueeze(1) )

trainloader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,
                            batch_size=32, shuffle=True)


/tmp/ipykernel_31631/2941146206.py:3: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180507909/work/torch/csrc/utils/tensor_numpy.cpp:189.)
  train_x = torch.Tensor(x[0:int(len(x)*0.7),])


In [11]:
import torch
from torch import optim
from Models.neural import STDConvModel
model = STDConvModel(3*5*24, 7, 12,stride1=2,stride2=2)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = model.to(device)
class reverseKLD(torch.nn.Module):
    """
    Class for creating a custom loss function, if desired.
    If you instead specify a standard loss function,
    you can remove or comment out this class.
    """
    def __init__(self):
        super(reverseKLD, self).__init__()
        self.loss = torch.nn.KLDivLoss()
    def forward(self, output, target):
        #return torch.mean((output - target)**2)
        return self.loss(target,output)
criterion = reverseKLD()
optimiser = optim.Adam(net.parameters())

In [12]:
def test_network(net,testloader, criterion):
    net.eval()
    total_images = 0
    #total_correct = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
    #        _, predicted = torch.max(outputs.data, 1)
            loss = criterion(output,labels)
            total_images += labels.size(0)
    #        total_correct += (predicted == labels).sum().item()

    model_accuracy = total_correct / total_images * 100
    #print('      Accuracy on {0} test images: {1:.2f}%'.format(
    #                            total_images, model_accuracy))
    print("Test Loss: ", loss)
    net.train()

In [13]:
print("Start training...")
for epoch in range(0,1):
    total_loss = 0
    total_timeframes = 0
    #total_correct = 0
    i = 0
    for batch in trainloader:           # Load batch
        batch_x, batch_y = batch 
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        preds = net(batch_x)             # Process batch
        #print(preds.shape,batch_y.shape)
        loss = criterion(preds, batch_y) # Calculate loss
        #print("Epoch:",epoch," Batch:",i," Loss:",loss)
        f = open("logs.txt", "a")
        f.write("Epoch:"+str(epoch)+" Batch:"+str(i)+" Loss:"+str(loss))
        f.write(str([x for x in net.parameters()]))
        f.close()
        #print([x for x in net.parameters()])
        i += 1
        optimiser.zero_grad()
        loss.backward()                 # Calculate gradients
        optimiser.step()                # Update weights

        output = preds.argmax(dim=1)

        total_loss += loss.item()
        total_timeframes += batch_y.size(0)
        #total_correct += output.eq(labels).sum().item()
    #scheduler.step()
    #model_accuracy = total_correct / total_images * 100
    print('epoch {0} total_correct: {1} loss: {2:.2f} acc: {3:.2f}'.format(
                epoch,-1, total_loss, -1) )
    
    #if epoch % 10 == 0:
    #    #if student.train_val_split < 1:
    #    test_network(net,testloader)
    #    torch.save(net.state_dict(), 'checkModel.pth')
    #    print("      Model saved to checkModel.pth")
    #    elapsed = time.time() - t
    #    print(elapsed)
    
#if student.train_val_split < 1:
#    test_network(net,testloader)
#torch.save(net.state_dict(), 'savedModel.pth')
#print("   Model saved to savedModel.pth")

Start training...


/home/sean/miniconda3/envs/qttrade/lib/python3.9/site-packages/torch/nn/functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


TypeError: write() argument must be str, not list

In [ ]:
import matplotlib.pyplot as plt
from torch import nn
#plt.plot(test_y)
#plt.show()
#plt.plot(net(test_x).detach().numpy())

print(test_y.shape)
#print(net(test_x).detach().numpy().shape)
#print(net(test_x).detach().numpy()[-20:])
#print(nn.BatchNorm1d(net.num_features+1)(nn.ReLU()(net.conv1d(test_x[-1:,]))))
import numpy as np
np.set_printoptions(threshold=10_000)
print(test_x[-1:,].shape)
print(test_x[-1:,:,0])
print(test_x[-1:,:,-1])
print(net.conv1d)
print(net.conv1d(test_x[-1:,]).shape)


torch.Size([19917, 1])
torch.Size([1, 7, 360])
tensor([[0.6035, 0.0265, 0.0762, 0.1438, 0.0334, 0.1990, 0.0094]])
tensor([[0.5991, 0.0331, 0.1230, 0.4267, 0.2158, 0.3956, 0.2113]])
Conv1d(7, 8, kernel_size=(12,), stride=(2,))
torch.Size([1, 8, 175])


In [ ]:
stub_x = torch.rand ((1,7,360))
print(stub_x[:,:,0])
print(stub_x[:,:,-1])
print(net.conv1d)
print(net.conv1d(stub_x))
newconv = nn.Conv1d(in_channels=7,out_channels=8,kernel_size=12,stride=2).to(device)
print(newconv)
print(newconv(stub_x))
print(net.conv1d.weight.data)


tensor([[0.9953, 0.1908, 0.7067, 0.2966, 0.3359, 0.3151, 0.0053]])
tensor([[0.2340, 0.2600, 0.5251, 0.7886, 0.3409, 0.1648, 0.1840]])
Conv1d(7, 8, kernel_size=(12,), stride=(2,))
tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]], grad_fn=<SqueezeBackward1>)
Conv1d(7, 8, kernel_size=(12,), stride=(2,))
tensor([[[ 0.5666,  0.4459,  0.0903,  ..., -0.0098,  0.4224,  0.2093],
         [-0.0355, -0.3532, -0.0667,  ...,  0.0538, -0.3279, -0.1290],
         [ 0.1829, -0.0448,  0.0867,  ..., -0.3430,  0.0703, -0.1879],
         ...,
         [ 0.3245,  0.3955,  0.3993,  ...,  0.3970,  0.4402,  0.1104],
         [-0.0392, -0.0503, -0.2970,  ..., -0.0571, -0.0177, -0.3347],
         [-0.2154,  0.0159, -0.2062,  ..., -0.2505, -0.1408, -0.2374]]],
       g

In [ ]:
import torch
from Models.neural import STDConvModel
m1 = STDConvModel(3*5*24, 2, 12,stride1=2,stride2=2)
m2 = STDConvModel(3*5*24, 2, 12,stride1=1,stride2=1)
from torchinfo import summary
print(m1(torch.zeros((20,2,5*24))).shape)
#print(summary(m1,(20,2,5*24)))
#print(summary(m2,(20,2,5*24)))

torch.Size([20, 1])
